In [123]:
#libraries
import pandas as pd


# Extract Data

In [124]:

PATH = r'C:\Users\ppuen\Documents\Entrevista Insurance Lunes\data.xlsx'
data = pd.read_excel(PATH)
data.head(1)

,Marital Status,AGE,Gender,Car Value,Years of No Claims Bonus,Annual Mileage,Payment Method,Acquisition Channel,Years of Tenure with Current Provider,Price,Actual Change in Price vs last Year,% Change in Price vs last Year,Grouped Change in Price,Renewed?
0,M,45.0,F,500.0,4.0,6000.0,Monthly,Inbound,4.0,289.4,-11.94,-0.0396,-0.05,0.0


# Transform Data

## Remove Unwanted Columns

In [125]:
data.drop(columns=['Marital Status','Car Value','Grouped Change in Price', 'Gender'], inplace=True)

## Policyholder age.

### EDA

In [ ]:
#rename column
data.rename(columns={"AGE":"age"}, inplace= True)
#check descriptive statistics
data.age.describe()

In [ ]:
#check discretization
print(pd.qcut(data.age, q = 20))
#apply
data['age_discrete'] = pd.qcut(data.age, q = 20)
#verifiy
data['age_discrete'].value_counts()
data['age'] = data.age.astype(int)

## Price , Actual Change in Price vs Last Year -- Expiring and Renewing Premium + Policy Renewed or Lapsed.


In [ ]:
#rename columns according to game
data.rename(columns={"Price":"renewing_premium","% Change in Price vs last Year":"%_chg_rnw_price_vs_actual", "Renewed?":'renewed'}, inplace =True)
#calculate expiring premium
data['expiring_premium'] = data['renewing_premium'] - data['Actual Change in Price vs last Year']
#Change Renewed value from 0 to -1, to calculate the non aggregated version of net revenue "net_revenue"
data['renewed_2'] = data['renewed']
data.loc[data['renewed'] == 0,'renewed_2'] = -1
data['renewed'].value_counts() 
#creating net revenue metric
data['net_revenue'] = data['renewed_2'] * data['renewing_premium']
#remove Actual Change in Price vs last Year	(not needed anymore)
data.drop(columns=['Actual Change in Price vs last Year'], inplace = True)
data.head(1)


## Acquisition Column

In [ ]:
data['Acquisition Channel'].value_counts()
#remove outbound category
data = data[data['Acquisition Channel'] != 'Outbound']
data['Acquisition Channel'].value_counts()

# Data Load

In [136]:
data.to_excel(r'C:\Users\ppuen\Documents\Entrevista Insurance Lunes\processed.xlsx') # to tableau

# Monthly Profiling

## Data Extract

In [12]:
# Choose an UK indicator
## https://www.ons.gov.uk/economy/nationalaccounts/uksectoraccounts/timeseries/npyh/ukea
## ESA:LEVEL:ASSET:FinCos: Insurance Premiums & Reserves
## Quarterly estimates of national product, income and expenditure, sector accounts and balance of payments.

#So first step get data from a file that is online and txt format and convert it into a table
##Learned something: With Python 3 urllib.urlretrieve() is considered legacy. no need for inventing headers anymore :)))
import pandas as pd
import requests
url ='https://www.ons.gov.uk/generator?format=csv&uri=/economy/nationalaccounts/uksectoraccounts/timeseries/npyh/ukea'
r = requests.get(url)
with open (rf"{PATH}\historical_data.txt", 'wb') as outfile:
    outfile.write(r.content)

historical_indicator_data = pd.read_csv(rf"{PATH}\historical_data.txt")
historical_indicator_data

,Title,ESA:LEVEL:ASSET:FinCos: Insurance Premiums & Reserves
0,CDID,NPYH
1,Source dataset ID,UKEA
2,PreUnit,NaN
3,Unit,NaN
4,Release date,31-03-2022
...,...,...
177,2020 Q4,573672
178,2021 Q1,393198
179,2021 Q2,349782
180,2021 Q3,348107


# Data Processing

In [36]:
import numpy as np
#Only Need the Quaterly Profiling So everything that on title has a "Q" is a good start
historical_data_ind_subset = historical_indicator_data[historical_indicator_data.Title.str.contains("Q") ]
#rename column titles
historical_data_ind_subset = historical_data_ind_subset.rename(columns={'Title':'QTR Date','ESA:LEVEL:ASSET:FinCos: Insurance Premiums & Reserves':'Ind_Value'})
#transform Ind_Value to numeric
historical_data_ind_subset.Ind_Value = pd.to_numeric(historical_data_ind_subset.Ind_Value)
# now get this table as profile to recreate the trend on ourt data
##get last value first which will be supposed to be the last point and coincides to the last point on the data I got  (assumption)
last_value = historical_data_ind_subset.Ind_Value.values[-1]
last_value
#Divide everything by last value
historical_data_ind_subset['profile'] = historical_data_ind_subset['Ind_Value']/ last_value

,QTR Date,Ind_Value,profile
42,1987 Q1,129099,0.364586
43,1987 Q2,130052,0.367277
44,1987 Q3,131001,0.369957
45,1987 Q4,131950,0.372637
46,1988 Q1,126410,0.356992
...,...,...,...
177,2020 Q4,573672,1.620094
178,2021 Q1,393198,1.110421
179,2021 Q2,349782,0.987811
180,2021 Q3,348107,0.983081


In [38]:
#This goes to Tableau
historical_data_ind_subset.to_csv(rf'{PATH}\Profile.csv')